In [23]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# read in csv 
df = pd.read_csv('data/df_final_2023_08_25')
# shape 
print("Shape: ", df.shape)

Shape:  (907485, 16)


In [24]:
print("Set the printing size of the pandas df: ")
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Set the printing size of the pandas df: 


In [25]:
print("Deleting the discontious SCC where period_index==-1")
print("Shape: ", df.shape)
df = df[df["period_index"] != -1]
print("Shape: ", df.shape)

Deleting the discontious SCC where period_index==-1
Shape:  (907485, 16)
Shape:  (671868, 16)


In [26]:
print("Shift scc to the first column")
# shift column 'scc' to first position
first_column = df.pop('scc')
# insert column using insert(position,column_name, first_column) function
df.insert(0, 'scc', first_column)

Shift scc to the first column


In [27]:
print("For convinent, make milkng_date the string")
milkng_date_only = df['milkng_date'].apply(lambda x: str(x))
df['milkng_date_only'] = milkng_date_only

For convinent, make milkng_date the string


# Pre-processing (de-trending, normalize cow, dataset chunking)

In [28]:
# ingore future warning
import warnings
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)
# surprass chain warning
pd.set_option('mode.chained_assignment', None)
#warnings.filterwarnings("ignore", message=r"Passing", category=SettingWithCopyWarning)

In [29]:
print("All the id of the herds: ")
print(len(df["hrd_ida"].unique().tolist()), df["hrd_ida"].unique().tolist())

All the id of the herds: 
40 [4, 50, 29, 34, 47, 9, 45, 31, 2, 30, 15, 19, 23, 42, 5, 7, 38, 40, 20, 36, 21, 39, 1, 8, 10, 18, 32, 25, 3, 22, 11, 41, 27, 6, 0, 14, 49, 37, 28, 26]


In [8]:
print("Rescale and bucket the scc value")

def conditions(i): 
    if i <= 50: return 50
    if i > 50 and i <= 100: return i
    if i > 100 and i <= 200: return i 
    if i > 200 and i <= 500: return i + 75
    
    if i > 500 and i <= 1000: return i + 125
    if i > 1000 and i <= 1500: return i + 250
    if i > 1500 and i <= 2000: return i + 500
    if i > 2000: return 3000



Rescale and bucket the scc value


### De-trending

### Chunking for time-series model

In [9]:
# 1. Given a dataset of a cow, find the start and end index of valid period (continous dim and the same period index) 

# 2. Given a valid period, chunk to X and y 

# 3. Use an overall function to loop through cows in a herd

In [10]:
# 1. Given a dataset of a cow, find the start and end index of valid period (continous dim and the same period index) 
""" 
func: 
    to find the start and end index of valid period 
args: 
    ls_dim: list[float] 
    ls_period_index: list[int] 
return: 
    ls_index: a list of start and end index of this cow: list[int]
"""
def find_valid_perod(ls_dim, ls_period_index): 
    
    # check the length of ls_dim 
    if len(ls_dim) <= 2: 
        return [0, len(ls_dim)]
        
    # init result and append the start index
    ls_index = [0]  
    
    # loop through the two input lists 
    for i in range(len(ls_dim)-1): 
        # if dim jumped or period index changed 
        if ((ls_dim[i+1] - 1) != ls_dim[i]) or (ls_period_index[i+1] != ls_period_index[i]): 
            # append new end index 
            ls_index.append(i) 
            
    # check for the last element  
    if (ls_dim[-1]-1 == ls_dim[-2]) and (ls_period_index[-1] == ls_period_index[-2]): 
        # append the last 
        ls_index.append(len(ls_dim))
        
    return ls_index
    

In [11]:
# 2. Given a valid period, chunk to X and y 
""" 
func: 
    given a valid period, create 
    dataset for seq2seq models X, y 
args: 
    valid_period: dataframe (the target (scc_std) has index 0!)
    X_len: int (length of input time period)
    y_len: int (length of output time period)
return: 
    don't return, keep appending ls_X, ls_y
"""
def chunk_to_X_y(valid_period, X_len, y_len, ls_X, ls_y):
    
    # check length 
    if valid_period.shape[0] < X_len + y_len: 
        return None

    # loop by the rows (start, end, step)
    for i in range(0, valid_period.shape[0]-X_len-y_len, y_len): 
        # append to X 
        ls_X.append(valid_period.iloc[i:i+X_len, :].to_numpy())
        # append to y 
        ls_y.append(valid_period.iloc[i+X_len:i+X_len+y_len, 0].to_numpy())
                    

In [12]:
# 3. Use an overall function to loop through cows in a herd
""" 
func: 
    loop through cows and chunking 
args: 
    df_stded: dataframe
    ls_predictors: list of variables keeped in prediction (list[str])
    X_len, y_len: the length of X and y (predictors and predicitons) (int, int)
return: 

"""
def chunk_herd(df_stded, ls_predictors):
    
    # list of cow id 
    ls_cow_id = df_stded['anm_ida'].unique().tolist() 

    # store valid_period 
    ls_df_period = []
    
    # looping cow id 
    for idx in ls_cow_id:
        
        # obtain cow data by index
        df_cow = df_stded[df_stded['anm_ida']==idx]

        """using func: """
        ls_index = find_valid_perod(df_cow['dim'].tolist(), df_cow['period_index'].tolist())

        # loop through ls_index for each valid period 
        for idx in range(len(ls_index)-1):
        
            # obtain a period from ls_index 
            valid_period = df_cow.iloc[ls_index[idx]:ls_index[idx+1], :]

            # keep only the predictor variables
            valid_period = valid_period[ls_predictors]
        
            # append to ls_df_period
            ls_df_period.append(valid_period)
            
    return ls_df_period

In [13]:
""" 
func: 
    to process a herd of data 
arg: 
    df_4: pd.Dataframe
    var_list_dim: list to correct for dim (list[str])
    var_list_date: list to correct for date (list[str])
    X_len: int
    y_len: int
    ls_predictors: list of vars keep (list[str])
    one_herd=True: boolean 
return: 
    ls_X: numpy 3d
    ls_y: numpy 2d 
    scc_mean: int
    scc_var: int
"""
def process_data_per_herd(df, var_list_dim, var_list_date, X_len, y_len, ls_predictors, one_herd=True):

    if one_herd:
        print("Put scc values in buckets to regulize regression model")
        """using func: """
        df['scc_bucket']=df['scc'].apply(conditions)
        print("check nan: ", df['scc_bucket'].isnull().sum())

    print("Correcting for dim...")
    for var_name in var_list_dim:
        # obtain the list of mean
        mean = df.groupby('dim').mean(numeric_only=True)[var_name]
        # convert to dictionary
        dict_ = mean.to_dict()
        # create new variable
        df[var_name] = df.apply(lambda x: x[var_name] - dict_[x['dim']], axis=1)

    print("Correcting for milkng_date_only...")
    for var_name in var_list_date:
        # obtain the list of mean
        mean = df.groupby('milkng_date_only').mean(numeric_only=True)[var_name]
        # convert to dictionary
        dict_ = mean.to_dict()
        # create new variable
        df[var_name] = df.apply(lambda x: x[var_name] - dict_[x['milkng_date_only']], axis=1)

    # we can pick if to standardize from here or not 
    if one_herd:
        print("Standardize SCC and storing")
        scc_var = df['scc_bucket'].std() 
        scc_mean = df['scc_bucket'].mean() 
        df["scc"] = (df['scc_bucket'] - scc_mean) / scc_var
        print("The mean and variance: ", scc_mean, scc_var)
    else: 
        scc_mean, scc_var = -1, -1
        print("Not standardizing scc per herd: ", scc_mean, scc_var)

    # standardization: except for scc where we have already stded
    for i in range(1, len(ls_predictors)): 
        df[ls_predictors[i]] = (df[ls_predictors[i]] - df[ls_predictors[i]].mean()) / df[ls_predictors[i]].std()

    """using func: """
    ls_X, ls_y = chunk_herd(df,  ls_predictors)
    
    return ls_X, ls_y, scc_mean, scc_var

# Sequence to sequence regression for each period

### Transformers

In [30]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers

In [31]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    
    layer = layers.MultiHeadAttention(num_heads=num_heads, key_dim=head_size)
    x = layer(x, x)
 
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    output_length,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    
    """"""
    outputs = layers.Dense(output_length)(x)
    return keras.Model(inputs, outputs)

In [32]:
""" 
func: 
    define model, split train&test, and evaluate
args: 
    ls_X: numpy 3d
    ls_y: numpy 2d
    scc_mean: int
    scc_var: int
    EPOCHS: int
return: 
    y_pred_value, y_test_value: numpy 2d
    SP, SE, AUC: int
"""
def train_and_evaluate_herd(ls_X, ls_y, X_len, y_len, scc_mean, scc_var, len_ls_predictors, EPOCHS):
    
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)
    
    print("Define model...")
    input_shape = ls_X[0].shape
    model = build_model(
        input_shape,
        head_size=64,
        num_heads=2,
        ff_dim=2,
        num_transformer_blocks=2,
        mlp_units=[32],
        output_length = y_len,
        mlp_dropout=0.25,
        dropout=0.25,
    )

    # split into training and testing
    training_ratio = 0.8
    # split and convert to numpy array format (required by tf)
    train_set = (np.array(ls_X[:int(training_ratio*len(ls_X))]), np.array(ls_y[:int(training_ratio*len(ls_y))]))
    test_set = (np.array(ls_X[int(training_ratio*len(ls_X)):]), np.array(ls_y[int(training_ratio*len(ls_y)):]))

    print("Check length of X and y: ", len(ls_X), len(ls_y))
    print("Check length of train and test: ", len(train_set[0]), len(test_set[0]))
    print("training_ratio: ", training_ratio)

    """using func: """
    valid_mae, model, histroy = fit_and_evaluate(model, train_set, test_set, learning_rate=0.02, epochs=EPOCHS)

    print("scc_mean, scc_var: ", scc_mean, scc_var)
    print("Average error: ", scc_mean * valid_mae)

    print("Making prediction...")
    y_pred = model.predict(test_set[0])
    y_pred_value =  (y_pred * scc_var) + scc_mean
    y_test_value = (test_set[1] * scc_var) + scc_mean
    
    # select scc_threshold based on herds
    #threshold_ls = [scc_mean-0.75*scc_var, scc_mean-0.5*scc_var, scc_mean-0.25*scc_var, scc_mean, scc_mean+0.25*scc_var,  scc_mean+0.5*scc_var, scc_mean+0.75*scc_var]
    #threshold_ls = [scc_mean-0.5*scc_var, scc_mean-0.4*scc_var, scc_mean-0.3*scc_var, scc_mean-0.2*scc_var, scc_mean-0.1*scc_var, scc_mean, scc_mean+0.1*scc_var, scc_mean+0.2*scc_var,  scc_mean+0.3*scc_var, scc_mean+0.4*scc_var, scc_mean+0.5*scc_var]
    threshold_ls = [scc_mean]
    performance_ls = [0.1, 0.1, 0.1]
    # set init AUC 
    AUC_max = 0.1 
    # looping for threshold 
    for i in threshold_ls:    
        """using func: regress_to_classify"""
        SP, SE, AUC = regress_to_classify(y_test_value, y_pred_value, scc_threshold=i)
        print("SP, SE, AUC: ", SP, SE, AUC)
        # pick by AUC
        if AUC >= AUC_max:
            # append to performance
            performance_ls = [SP, SE, AUC]
            # reset AUC_max 
            AUC_max = AUC
    
    # end looping 
    print()
    print("Final SP, SE, AUC: ", performance_ls)
    
    return y_pred_value, y_test_value, SP, SE, AUC

In [33]:
# extra code – defines a utility function we'll reuse several time
"""
func: 
    fit_and_evaluate
args: 
    model: tf model
    train_set: tuple(X_train, y_train)
    valid_set: tuple(X_test, y_test)
    learning_rate: float
    epochs: int 
"""
def fit_and_evaluate(model, train_set, valid_set, learning_rate, epochs=10):
    
    # set the early stopping 
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        monitor="val_mae", patience=5, restore_best_weights=True)
    
    # set optimizers
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    
    # compile the model
    model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=["mae"])
    
    # fitting the model 
    history = model.fit(
                    x = train_set[0], 
                    y = train_set[1],
                    batch_size=128,
                    validation_data=(valid_set[0], valid_set[1]), 
                    epochs=epochs,
                    callbacks=[early_stopping_cb])
    
    # compute the validation loss
    valid_loss, valid_mae = model.evaluate(valid_set[0], valid_set[1])
    
    return valid_mae, model, history

In [34]:
""" 
func: 
    to evaluate regression by 
    converting to classification problem 
args: 
    y_test_value: numpy (number of period, length)
    y_pred_value: numpy (number of period, length)
    scc_threshold: the scc threshold set to pick out mastitis in the predicion set 
    buffer = 5: int (the delay of mastitis occurence)
return: 
    SP, SE, AUC
"""
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
def regress_to_classify(test, pred, scc_threshold, buffer = 5): 
    
    # set result y_pred as the num of rows (example)
    y_pred = [0 for i in range(pred.shape[0])]
    y_test = [0 for i in range(pred.shape[0])]

    # loop rows 
    for i in range(pred.shape[0]): 
        one_pred = pred[i, -buffer:]
        # set 1 in y_pred base on the threshold 
        if max(one_pred) >= scc_threshold: y_pred[i] = 1
        one_test = test[i, -buffer:]
        # set 1 in y_test base on the threshold 
        if max(one_test) >= 2500: y_test[i] = 1
    
    """using sklearn"""
    try:
        # this func requie y_test to have at least two classes
        auc = roc_auc_score(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        sp = tn / (tn + fp)
        se = tp / (tp + fn)
    except ValueError:
        print("Invalid threshold")
        sp, se, auc = 0.1, 0.1, 0.1

    print("sp, se, auc: ", sp, se, auc)
    return sp, se, auc

# Combine data-processing and prediction for herds

### Training on individual herds

### Training all the herds together

In [35]:
""" 
func: 
    the final function from dataframe to X, y 
args: 
    df: pd.Dataframe (raw df) 
    var_list_dim: list[str] 
    var_list_date: list[str] 
    X_len: int (input period) 
    y_len: int (output period) 
    ls_predictors: list[str]
return: 
    ls_X_ls: np.array (examples, input period, variables)
    ls_y_ls: np.array (examples, input period)
    scc_mean, scc_var: float 
"""
def process_all_herd(df, var_list_dim, var_list_date, X_len, y_len, ls_predictors):

    """using func: conditions"""
    df['scc']=df['scc'].apply(conditions)
    print("check nan: ", df['scc'].isnull().sum())

    print("Correcting for dim...")
    for var_name in var_list_dim:
        # obtain the list of mean
        mean = df.groupby('dim').mean(numeric_only=True)[var_name]
        # convert to dictionary
        dict_ = mean.to_dict()
        # create new variable
        df[var_name] = df.apply(lambda x: x[var_name] - dict_[x['dim']], axis=1)

    print("Correcting for milkng_date_only...")
    for var_name in var_list_date:
        # obtain the list of mean
        mean = df.groupby('milkng_date_only').mean(numeric_only=True)[var_name]
        # convert to dictionary
        dict_ = mean.to_dict()
        # create new variable
        df[var_name] = df.apply(lambda x: x[var_name] - dict_[x['milkng_date_only']], axis=1)

    # we can pick if to standardize from here or not
    scc_var = df['scc'].std() 
    scc_mean = df['scc'].mean() 

    # standardization:
    for i in range(len(ls_predictors)): 
        df[ls_predictors[i]] = (df[ls_predictors[i]] - df[ls_predictors[i]].mean()) / df[ls_predictors[i]].std()

    """using func: """
    ls_df_valid = chunk_herd(df,  ls_predictors)

    return ls_df_valid, scc_mean, scc_var

In [36]:
# init list with variables needed to be corrected by dim 
var_list_dim = ["mk_wgt", "min_time", "milk_flow_max", "ft_pcnt"]
# init list with variables needed to be corrected by dim 
var_list_date = ["milkng_temp"]
# a list of predictor variables: 1 target + 6 variables
ls_predictors = ['scc', 'mk_wgt', 'min_time', 'milk_flow_max', 'ft_pcnt', 'milkng_temp']
# X length is the length of input X 
X_len = 21
# y length is the length of output y 
y_len = 5

"""using func: """
# to prevent in-place change 
df_new = df
ls_df_valid, scc_mean, scc_var = process_all_herd(df_new, var_list_dim, var_list_date, X_len, y_len, ls_predictors)

check nan:  0
Correcting for dim...
Correcting for milkng_date_only...


# Experimenting

In [37]:
# init result_ls 
result_ls = [] 
# X length is the length of input X, y length is the length of output y 
ls_X_len_y_len = [(21, 5), (28, 5), (35, 5), (21, 8), (28, 8), (35, 8), (21, 10), (28, 10), (35, 10)]

for pair in ls_X_len_y_len:
    X_len, y_len = pair[0], pair[1]
    print("========================================")
    print("X_len, y_len: ", X_len, y_len)
    
    # init X, y
    ls_X, ls_y = [], [] 
    # looping ls_df_valid (list[df])
    for i in range(len(ls_df_valid)): 
        # obtain period
        valid_period = ls_df_valid[i]
        """using func: """
        chunk_to_X_y(valid_period, X_len, y_len, ls_X, ls_y)
        
    # here 
    """using func: train_and_evaluate_herd"""
    _, _, SP, SE, AUC = train_and_evaluate_herd(ls_X, ls_y, X_len, y_len, scc_mean, scc_var, len(ls_predictors), EPOCHS=10)
    result_ls.append([SP, SE, AUC])

print(result_ls)

X_len, y_len:  21 5
Define model...
Check length of X and y:  81403 81403
Check length of train and test:  65122 16281
training_ratio:  0.8
Epoch 1/10
509/509 [==============================] - 39s 70ms/step - loss: 0.1975 - mae: 0.3818 - val_loss: 0.1724 - val_mae: 0.3295
Epoch 2/10
509/509 [==============================] - 35s 69ms/step - loss: 0.1715 - mae: 0.3278 - val_loss: 0.1688 - val_mae: 0.3191
Epoch 3/10
509/509 [==============================] - 35s 68ms/step - loss: 0.1683 - mae: 0.3209 - val_loss: 0.1660 - val_mae: 0.3155
Epoch 4/10
509/509 [==============================] - 35s 69ms/step - loss: 0.1671 - mae: 0.3194 - val_loss: 0.1654 - val_mae: 0.3178
Epoch 5/10
509/509 [==============================] - 34s 68ms/step - loss: 0.1659 - mae: 0.3172 - val_loss: 0.1641 - val_mae: 0.3188
Epoch 6/10
509/509 [==============================] - 36s 72ms/step - loss: 0.1652 - mae: 0.3158 - val_loss: 0.1645 - val_mae: 0.3343
Epoch 7/10
509/509 [==============================] - 35

175/175 [==============================] - 19s 111ms/step - loss: 0.1907 - mae: 0.3719 - val_loss: 0.1757 - val_mae: 0.3511
Epoch 3/10
175/175 [==============================] - 19s 111ms/step - loss: 0.1872 - mae: 0.3600 - val_loss: 0.1726 - val_mae: 0.3473
Epoch 4/10
175/175 [==============================] - 21s 118ms/step - loss: 0.1832 - mae: 0.3517 - val_loss: 0.1730 - val_mae: 0.3453
Epoch 5/10
175/175 [==============================] - 20s 114ms/step - loss: 0.1815 - mae: 0.3466 - val_loss: 0.1713 - val_mae: 0.3311
Epoch 6/10
175/175 [==============================] - 19s 110ms/step - loss: 0.1797 - mae: 0.3422 - val_loss: 0.1717 - val_mae: 0.3338
Epoch 7/10
175/175 [==============================] - 19s 111ms/step - loss: 0.1780 - mae: 0.3386 - val_loss: 0.1694 - val_mae: 0.3258
Epoch 8/10
175/175 [==============================] - 20s 114ms/step - loss: 0.1771 - mae: 0.3360 - val_loss: 0.1657 - val_mae: 0.3246
Epoch 9/10
175/175 [==============================] - 20s 112ms/st

In [1]:
result_ls = [[0.8738897596656218, 0.8121775025799793, 0.8430336311228006], [0.8769747386094787, 0.8331318016928658, 0.8550532701511722], [0.8723765294275252, 0.834061135371179, 0.853218832399352], [0.8698600645855759, 0.7906574394463668, 0.8302587520159713], [0.8852811118130133, 0.7860082304526749, 0.8356446711328441], [0.8904882029974773, 0.7929411764705883, 0.8417146897340327], [0.8812361419068736, 0.7734204793028322, 0.8273283106048529], [0.8677911646586345, 0.7791878172588832, 0.8234894909587588], [0.865781990521327, 0.8444444444444444, 0.8551132174828857]]

### Plot the results 

In [41]:
""" 
func: 
    to write the label on plots 
args: 
    dates, temp: list of x and y in the plot 
    plt: the plot object
return: 
    annotated plot
"""
import matplotlib.pyplot as plt
import numpy as np
def write_label(dates, temp, plt): 
    for x, y in zip(dates, temp):
        label = y
        plt.annotate(label, (x, y),
                     xycoords="data",
                     textcoords="offset points",
                     xytext=(0, 10), ha="center")

""" 
func: 
    to plot for a model's result 
args: 
    log_ls: list[list[float]] the results of this model
    model_name: str
    path: str
return: 
    name_ls: list[str]
"""
def plot_one_model(log_ls, model_name, path):
    
    from matplotlib.pyplot import figure



    # init methods
    method_str = [' 21 days', ' 28 days', '35 days']
    # init method id (0, 1, 2)
    idx_method_ls = [0, 1, 2]
    # total num of methods
    shrink_method = 3 
    # init name 
    name_ls = []

    # looping for idx_method 
    for idx_method in idx_method_ls: 

        # init list for SP, SE, AUC 
        SP, SE, AUC = [], [], [] 

        # loop the list 
        for i in range(idx_method, len(log_ls), shrink_method):
            # specificity
            sp = round(log_ls[i][0]*100, 2)
            SP.append(sp)
            # sensitifity
            se = round(log_ls[i][1]*100, 2)
            SE.append(se)
            # auc
            auc = round(log_ls[i][2]*100, 2)
            AUC.append(auc)   

        # reserse the list
        SP, SE, AUC = SP[::-1], SE[::-1], AUC[::-1]
        x_axis = ['5 days', '3 days', '1 days']

        plt.plot(x_axis, SP, label = 'specificity', color="indigo", marker = 'o')
        """using func: """
        write_label(x_axis, SP, plt)
        plt.plot(x_axis, SE, label = 'sensitivity', color="indigo", marker = 's')
        """using func: """
        write_label(x_axis, SE, plt)
        plt.plot(x_axis, AUC, label = 'AUC', color="forestgreen", marker = '^')
        """using func: """
        write_label(x_axis, AUC, plt)
        plt.legend() 
        plt.title(model_name + " with input length: " + method_str[idx_method])
        plt.xlabel("Days before the threshold")
        
        # saving 
        name = model_name + "_" + method_str[idx_method]
        name_ls.append(name)
        plt.gcf().set_size_inches(5, 5)
        plt.savefig(path+name, dpi=200)
        plt.close()
        
    return name_ls


In [42]:
name_ls = plot_one_model(result_ls, 'Transformers', 'plot/Results_regression_new/')

In [43]:
""" 
func: 
    concate plots 
args: 
    name_ls_log, name_ls_tree, name_ls_mlp: list[str]
return: 
    a plot
"""
def concat_plots(name_ls, path, name):
    
    temp = name_ls
    figa = plt.imread(path + temp[0] + '.png')
    figb = plt.imread(path + temp[1] + '.png')
    figc = plt.imread(path + temp[2] + '.png')
    figd = np.concatenate((figa,figb), axis=0)
    fige = np.concatenate((figd,figc), axis=0) 
                   
    fname = path + name + ".png"
    plt.imsave(fname, fige)

In [44]:
concat_plots(name_ls, 'plot/Results_regression_new/', 'transformers_longer')